In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder as SklearnOneHotEncoder
from statsmodels.regression.linear_model import OLS

In [2]:
class OneHotEncoder(SklearnOneHotEncoder):
    def __init__(self, **kwargs):
        super(OneHotEncoder, self).__init__(**kwargs)
        self.fit_flag = False

    def fit(self, X, **kwargs):
        out = super().fit(X)
        self.fit_flag = True
        return out

    def transform(self, X, **kwargs):
        sparse_matrix = super(OneHotEncoder, self).transform(X)
        new_columns = self.get_new_columns(X=X)
        d_out = pd.DataFrame(sparse_matrix.toarray(), columns=new_columns, index=X.index)
        return d_out

    def fit_transform(self, X, **kwargs):
        self.fit(X)
        return self.transform(X)

    def get_new_columns(self, X):
        new_columns = []
        for i, column in enumerate(X.columns):
            j = 0
            while j < len(self.categories_[i]):
                new_columns.append(f'{column}_<{self.categories_[i][j]}>')
                j += 1
        return new_columns

### Линейная регрессия для оценки влияния категориального фактора на метрику

Мы применим подход dummy-переменных, который преобразовывает каждое значение категориальной фичи в отдельную колонку содержащую 0, если в этой строке этой категории нет, и 1, если эта категория присутствует.

Почему же линейная регрессия способна хорошо себя показать?

Для начала распишем, что такое линейная регрессия:

$$
Y = {\alpha} * X + {\beta}
$$

Если подойти чуть строже, то линейная регрессия оценивает математическое ожидание Y, при условии X, т.е:

$$
E[Y | X] = {\alpha} * X + {\beta}
$$


В нашем случае переменная преобразовывается из бинарной в категориальную, т.е. мы получаем оценку матожидания при условии присутствия или отсутствия категории. Т.е. коэффициент перед нашей dummy-переменной мы можем трактовать как прирост к метрике, если у нас "включена" данная категория.

$$
E[Y | X=1] = {\alpha} + {\beta}
$$

$$
E[Y | X=0] = {\beta}
$$


Простой пример, который можно здесь привести - это аналитика влияния промо-акций на метрики, например на средний чек. Промо-акция - это категориальная переменная, которую мы преобразуем в dummy-переменную. Этот пример, кстати, позволит понять, что именно означает коэффициент ${\alpha}$ в модели линейной регрессии, - а он означает прирост к среднему чеку, если на заказе было применено промо. А ${\beta}$ - это просто средний чек по всем заказам без промо.

Давайте построи подобную модель на реальных данных и обсудим её преимущества.

In [13]:
df = pd.read_csv('one_promo_df.csv')

In [16]:
df.promo_type.unique()

array(['no_promo', 'SALE15'], dtype=object)

In [17]:
df.promo_type

0        no_promo
1        no_promo
2        no_promo
3        no_promo
4        no_promo
           ...   
99964    no_promo
99965    no_promo
99966    no_promo
99967    no_promo
99968    no_promo
Name: promo_type, Length: 99969, dtype: object

In [18]:
encoder = OneHotEncoder()

In [19]:
encoder.fit_transform(df[['promo_type']])

,promo_type_<SALE15>,promo_type_<no_promo>
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
99964,0.0,1.0
99965,0.0,1.0
99966,0.0,1.0
99967,0.0,1.0


In [9]:
X = encoder.fit_transform(df[['promo_type']]).drop('promo_type_<no_promo>', axis=1).assign(aov=1)

Y = df['order_value']

In [10]:
estimator = OLS(Y, X).fit()

In [11]:
print(estimator.summary())

                            OLS Regression Results                            
Dep. Variable:            order_value   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.166
Date:                Tue, 19 Jul 2022   Prob (F-statistic):              0.280
Time:                        10:37:10   Log-Likelihood:            -3.5255e+05
No. Observations:               99969   AIC:                         7.051e+05
Df Residuals:                   99967   BIC:                         7.051e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
promo_type_<SALE15>    -0.0938    